# Decision Trees Alogorithms - Impurity Measures: ID3, CART, and C4.5; Entropy, Gini Index, and Classification Error

## Why are decision trees important?

### They're easily understood

Decision trees are easy to understand and they capture some very fundamental and important concepts in machine learning and expert systems. The process of classification is self-evident and easy to implement.

### They are computationally efficient

Straight-forwardly, they don't require massive amounts of computational power, and this is a huge bonus. If you can use a classification tree, do. The computational complexity is proportional to the size of the dataset, nothing more.

### They are able to generalize given an appropriate training set

If the instances are descrived in terms of features that are well-correlated with the target class, then the model will generalize and be able to classify unobserved instances.

## Decision Criteria

### Splitting Criteria

All decision tree algorithms use a criteria for splitting nodes in the tree. Typically trees split in a dichotomous fashion, with two edges/branches splitting from each internal node.

The measure of *impurity*, known as *information entropy* or *information gain*, is the criteria for splitting the tree at a node. The goal of splitting criteria is to reduce the impurity of a node and maximize the amount of information gained at each node in the process of making a decision. The nodes closest to the root will give the greatest amount of information, and as the process winds down to the leaf nodes, the final little bits of information make the final decision.

These splitting criteria are measured and defined in terms of the target-class distribution amongst the instances in the dataset.

Some of the most ubiquitous measures of impurity include:
1. Entropy
2. Gini Index
3. Classification Error
4. Information Gain
5. Gain Ratio
6. Twoing Criteria

#### Almost Twins: Entropy versus Gini Index

Gini Impurity and Entropy are so similar that they are nearly the same thing, but they do have some distinctions. They come out of different fields of science. They can be used interchangably in many cases, but for a small percentage of use cases it is important to know which to choose.

Making this choice comes down to the nature of your data.

#### Entropy is for attributes that occur in classes

It is a measure of impurity. It may seem a little slower to compute up front, but remembering that logarithms allow us to turn multiplication into addition reveals why entropy is such an important concept. Imagine a dataset with 500,000 entries. If we had to multiply the probability of 500,000 instances, each of which is between 0 and 1, we're going to get an absurdly small number with many, many, many zeroes. 


The general formula for Entropy is given

$$Entropy(x) = -\sum_{i=1}^{n} p_{i}(x)log_{2}{(p_{i}(x))}$$

where:
 - $x$ is an object from the dataset
 - $p_i$ is the probability of that object (an object of the $i$th class) appearing
 - $n$ is the number of classes

#### Gini Index is for continuous attributes
*Aka Gini Coefficient and Gini Ratio*

Consider an experiment with $n$ possible output classes. Class $i$ has a probability of occurring equal to $p(i|x)$ where $i = 1,..n$.


Take two successive observations and notice that the following is true:
 - the probability of observing identical outcomes of the same class $i$ is $p^2(i|x)$
 - the porbability of two identical outcomes independent of their class is $\sum_{i = 1}^{n}p^2(i|x)$
 - it follows immediately that the probability of obtaining two *different* outputs is $1 - \sum_{i = 1}^{n}p^2(i|x)$
 
 
 The Gini impurity is simple the probability of observing two different classes in a row and it is given $$G(x) = 1 - \sum_{i = 1}^{n}p^2(i|x)$$

#### Classification Error

$$ClassificationError(x) = 1 - max[p(i|x)]$$

#### Information Gain

$$InfoGain(\nabla) = Entropy(parent) - Entropy(child)$$

#### Gain Ratio

$$RainRatio = \frac{InfoGain(\nabla)}{Entropy}$$

#### Twoing Criteria

The Gini Index may be problematic in handling problems where the domain of the target attribute is relatively wide. In this case you can use a binary criterion called twoing criteria it is defined as:

$$Twoing(x)= \frac{P_L P_R}{4}(\sum{abs(p(i|x_L) - p(i|x_R))})^2$$

### Stopping Criteria

The stopping criteria determines when the splitting (aka growing) phase stops. The rules for stopping criteria are as such:

1. All instances in the trianing set are of a single class at the current node.
2. The tree is at maximum depth.
3. The number of cases (not to be confused with instances) in the terminal node is fewer than the minimum number of cases for its parent nodes.
4. If the given node were to split then one of its child nodes would be less than the minimum number of cases for child nodes. *(This rule exists to prevent over-fitting.)*
5. The best splitting criteria is not greater than a defined threshold.

In [1]:
import arff

In [3]:
# data packages
import os, arff, numpy as np, pandas as pd

# learning package
import sklearn

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style('monokai')

In [7]:
os.getcwd()

'/home/torchl/Documents/GitHub/Notebooks/data-science-and-research-skills'

In [22]:
dataset = arff.load(open('/home/torchl/Documents/WekaDatasets/weather.nominal.arff'), 'rb')
data = np.asarray(dataset['data'])

In [27]:
dataset['attributes']

[('outlook', ['sunny', 'overcast', 'rainy']),
 ('temperature', ['hot', 'mild', 'cool']),
 ('humidity', ['high', 'normal']),
 ('windy', ['TRUE', 'FALSE']),
 ('play', ['yes', 'no'])]

In [30]:
data

array([[0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [2, 1, 0, 1, 0],
       [2, 2, 1, 1, 0],
       [2, 2, 1, 0, 1],
       [1, 2, 1, 0, 0],
       [0, 1, 0, 1, 1],
       [0, 2, 1, 1, 0],
       [2, 1, 1, 1, 0],
       [0, 1, 1, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 1, 1, 0],
       [2, 1, 0, 0, 1]])

In [33]:
col_names = [i[0] for i in dataset['attributes']]

In [34]:
col_names

['outlook', 'temperature', 'humidity', 'windy', 'play']

In [36]:
dataset = pd.DataFrame(data)

In [38]:
dataset.columns = col_names

In [75]:
dataset

,outlook,temperature,humidity,windy,play
0,0,0,0,1,1
1,0,0,0,0,1
2,1,0,0,1,0
3,2,1,0,1,0
4,2,2,1,1,0
5,2,2,1,0,1
6,1,2,1,0,0
7,0,1,0,1,1
8,0,2,1,1,0
9,2,1,1,1,0


In [ ]:
target = dataset['play']